In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.getcwd()

'C:\\Users\\adb\\iCloudDrive\\Adnan PC\\Data Science\\Jupyter NB'

In [3]:
from EDA import eda_df

In [4]:
train = pd.read_csv(r'.\kaggle\housing\house-prices-advanced-regression-techniques\train.csv')

In [5]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
eda_df(train)

=================Shape============================
DataFrame Shape:  (1460, 81)
=================INFO============================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType  

44 . RoofMatl ['CompShg' 'WdShngl' 'Metal' 'WdShake' 'Membran' 'Tar&Grv' 'Roll'
 'ClyTile']
45 . HouseStyle ['2Story' '1Story' '1.5Fin' '1.5Unf' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin']
46 . Condition2 ['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']
47 . PoolArea [  0 512 648 576 555 480 519 738]
48 . BedroomAbvGr [3 4 1 2 0 5 6 8]
49 . SaleType ['WD' 'New' 'COD' 'ConLD' 'ConLI' 'CWD' 'ConLw' 'Con' 'Oth']
50 . Condition1 ['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']
51 . OverallCond [5 8 6 7 4 2 3 9 1]
52 . OverallQual [ 7  6  8  5  9  4 10  3  1  2]
53 . TotRmsAbvGrd [ 8  6  7  9  5 11  4 10 12  3  2 14]
54 . MoSold [ 2  5  9 12 10  8 11  4  1  7  3  6]


In [7]:
train_X = train.iloc[:,:-1]
train_y = train.iloc[:,-1]

In [8]:
num_col = list(train_X.describe().columns)
cat_col = list(set(train_X.columns).difference(num_col))

In [9]:
print(len(train_X.columns),len(cat_col),len(num_col))

80 43 37


In [10]:
def test_df(df):
    assert(len(df.columns)==80)

In [11]:
from sklearn.preprocessing import FunctionTransformer
new_col_list = []
dropped_col_list = []
df = train_X.copy()
na_count_series = df.isna().mean()
dropped_col_list = na_count_series[na_count_series>.4].index.to_list()

        
def col_remove(df):
    global new_col_list
    global dropped_col_list
    for col in dropped_col_list:
        if col in df.columns:
            df = df.drop(columns=col,axis=1)
    new_col_list = list(set(df.columns).difference(dropped_col_list))
    return df

In [12]:
def fix_na(X):
    X = X.fillna(X.mode().iloc[0])
    return np.c_[X]

In [13]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

num_Pipeline = Pipeline([
#     ('attr_rem',FunctionTransformer(col_remove,validate=False)),
    ('fill_na',FunctionTransformer(fix_na,validate=False)),
    ('imputer',SimpleImputer(strategy='median')),
    ('std_scaler',StandardScaler()),
])


In [14]:
from sklearn.preprocessing import OneHotEncoder
cat_Pipeline = Pipeline([
#     ('attr_rem',FunctionTransformer(col_remove,validate=False)),
    ('fill_na',FunctionTransformer(fix_na,validate=False)),    
    ('cat_enc',OneHotEncoder())
])


In [15]:
from sklearn.compose import ColumnTransformer

full_Pipeline = ColumnTransformer([
    ('num',num_Pipeline,num_col),
    ('cat',cat_Pipeline,cat_col),
])

housing_preproc = full_Pipeline.fit_transform(train_X)

In [16]:
housing_preproc.size

116800

## Modelling

### Train/Test split

In [17]:
X = housing_preproc
y= train_y

In [18]:
from sklearn.model_selection import train_test_split


In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

lr = LinearRegression()
cv = cross_val_score(lr,X,y)
print(cv)
print(cv.mean())

[0.81723043 0.82177606 0.80300543 0.88821732 0.64508627]
0.7950631019439506


In [20]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver="liblinear", random_state=42)
cv = cross_val_score(log_reg,X,y)
print(cv)
print(cv.mean())

C:\Users\adb\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


[0.0239726  0.00342466 0.01369863 0.01712329 0.00342466]
0.012328767123287671


In [21]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
cv = cross_val_score(tree_reg,X,y)
print(cv)
print(cv.mean())

[0.70579727 0.74042153 0.83738121 0.75509043 0.65520805]
0.738779699404774


In [22]:
scores = cross_val_score(tree_reg, X, y,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [23]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

Scores: [36132.31461833 40371.0448103  36516.10875213 43665.81705917
 38155.74187082 28367.12891301 34140.60112733 34957.0746815
 48492.81398084 36238.879841  ]
Mean: 37703.75256544255
Standard deviation: 5222.139383896559


In [24]:
lin_scores = cross_val_score(lr, X, y,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [24110.54335233 35253.24867128 23310.63175753 41612.96532634
 30574.53565328 43472.56647212 24163.65138133 22662.15218288
 66662.72690708 22248.26976286]
Mean: 33407.129146703686
Standard deviation: 13415.653627085188


In [25]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten

Using TensorFlow backend.
C:\Users\adb\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\adb\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\adb\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\adb\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Fu

In [26]:
model = Sequential()

# Input Layer
model.add(Dense(128,kernel_initializer='normal',input_dim=X.shape[1],activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(1,kernel_initializer='normal',activation='linear'))

model.compile(loss='mean_absolute_error',optimizer='adam',metrics=['mean_absolute_error'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               37120     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 70,401
Trainable params: 70,401
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(X,y,epochs=500,batch_size=32,validation_split=0.2)


Train on 1168 samples, validate on 292 samples
Epoch 1/500
1168/1168 [==============================] - 0s 184us/step - loss: 180570.1729 - mean_absolute_error: 180570.1875 - val_loss: 182161.2735 - val_mean_absolute_error: 182161.2656
Epoch 2/500
1168/1168 [==============================] - 0s 73us/step - loss: 180207.5599 - mean_absolute_error: 180207.5625 - val_loss: 181326.8104 - val_mean_absolute_error: 181326.8125
Epoch 3/500
1168/1168 [==============================] - 0s 69us/step - loss: 178357.7352 - mean_absolute_error: 178357.7344 - val_loss: 178209.1929 - val_mean_absolute_error: 178209.1875
Epoch 4/500
1168/1168 [==============================] - 0s 75us/step - loss: 173143.5201 - mean_absolute_error: 173143.5000 - val_loss: 170805.2568 - val_mean_absolute_error: 170805.2656
Epoch 5/500
1168/1168 [==============================] - 0s 70us/step - loss: 162400.1107 - mean_absolute_error: 162400.0938 - val_loss: 157008.6616 - val_mean_absolute_error: 157008.6562
Epoch 6/500

Epoch 44/500
1168/1168 [==============================] - 0s 68us/step - loss: 15237.4060 - mean_absolute_error: 15237.4043 - val_loss: 19664.9007 - val_mean_absolute_error: 19664.9023
Epoch 45/500
1168/1168 [==============================] - 0s 70us/step - loss: 15157.8862 - mean_absolute_error: 15157.8867 - val_loss: 19653.8315 - val_mean_absolute_error: 19653.8301
Epoch 46/500
1168/1168 [==============================] - 0s 67us/step - loss: 15047.4764 - mean_absolute_error: 15047.4756 - val_loss: 19603.9361 - val_mean_absolute_error: 19603.9336
Epoch 47/500
1168/1168 [==============================] - 0s 72us/step - loss: 15054.6878 - mean_absolute_error: 15054.6885 - val_loss: 19505.1790 - val_mean_absolute_error: 19505.1777
Epoch 48/500
1168/1168 [==============================] - 0s 71us/step - loss: 14909.8517 - mean_absolute_error: 14909.8506 - val_loss: 19497.9693 - val_mean_absolute_error: 19497.9668
Epoch 49/500
1168/1168 [==============================] - 0s 73us/step - lo

1168/1168 [==============================] - 0s 73us/step - loss: 13458.3374 - mean_absolute_error: 13458.3398 - val_loss: 18363.7512 - val_mean_absolute_error: 18363.7520
Epoch 89/500
1168/1168 [==============================] - 0s 77us/step - loss: 13462.8120 - mean_absolute_error: 13462.8125 - val_loss: 18326.2408 - val_mean_absolute_error: 18326.2402
Epoch 90/500
1168/1168 [==============================] - 0s 66us/step - loss: 13443.2670 - mean_absolute_error: 13443.2676 - val_loss: 18305.3345 - val_mean_absolute_error: 18305.3359
Epoch 91/500
1168/1168 [==============================] - 0s 65us/step - loss: 13426.4406 - mean_absolute_error: 13426.4404 - val_loss: 18334.2962 - val_mean_absolute_error: 18334.2969
Epoch 92/500
1168/1168 [==============================] - 0s 67us/step - loss: 13454.2966 - mean_absolute_error: 13454.2979 - val_loss: 18352.3193 - val_mean_absolute_error: 18352.3184
Epoch 93/500
1168/1168 [==============================] - 0s 67us/step - loss: 13394.256

1168/1168 [==============================] - 0s 68us/step - loss: 12982.9282 - mean_absolute_error: 12982.9297 - val_loss: 17961.7571 - val_mean_absolute_error: 17961.7539
Epoch 133/500
1168/1168 [==============================] - 0s 71us/step - loss: 12985.2850 - mean_absolute_error: 12985.2861 - val_loss: 17972.3687 - val_mean_absolute_error: 17972.3691
Epoch 134/500
1168/1168 [==============================] - 0s 72us/step - loss: 12947.3163 - mean_absolute_error: 12947.3154 - val_loss: 17957.1867 - val_mean_absolute_error: 17957.1875
Epoch 135/500
1168/1168 [==============================] - 0s 70us/step - loss: 12960.7489 - mean_absolute_error: 12960.7490 - val_loss: 17952.4698 - val_mean_absolute_error: 17952.4688
Epoch 136/500
1168/1168 [==============================] - 0s 72us/step - loss: 12933.9339 - mean_absolute_error: 12933.9346 - val_loss: 17997.8042 - val_mean_absolute_error: 17997.8047
Epoch 137/500
1168/1168 [==============================] - 0s 78us/step - loss: 1294

1168/1168 [==============================] - 0s 79us/step - loss: 12735.4571 - mean_absolute_error: 12735.4561 - val_loss: 17948.6732 - val_mean_absolute_error: 17948.6738
Epoch 177/500
1168/1168 [==============================] - 0s 85us/step - loss: 12724.3456 - mean_absolute_error: 12724.3467 - val_loss: 17978.3002 - val_mean_absolute_error: 17978.2988
Epoch 178/500
1168/1168 [==============================] - 0s 76us/step - loss: 12761.7334 - mean_absolute_error: 12761.7324 - val_loss: 17962.1763 - val_mean_absolute_error: 17962.1758
Epoch 179/500
1168/1168 [==============================] - 0s 73us/step - loss: 12747.2285 - mean_absolute_error: 12747.2266 - val_loss: 17981.7251 - val_mean_absolute_error: 17981.7246
Epoch 180/500
1168/1168 [==============================] - 0s 71us/step - loss: 12761.7300 - mean_absolute_error: 12761.7285 - val_loss: 18000.5808 - val_mean_absolute_error: 18000.5820
Epoch 181/500
1168/1168 [==============================] - 0s 72us/step - loss: 1273

...

1168/1168 [==============================] - 0s 70us/step - loss: 12352.3391 - mean_absolute_error: 12352.3389 - val_loss: 17986.0569 - val_mean_absolute_error: 17986.0566
Epoch 309/500
1168/1168 [==============================] - 0s 71us/step - loss: 12341.3684 - mean_absolute_error: 12341.3682 - val_loss: 17984.2054 - val_mean_absolute_error: 17984.2051
Epoch 310/500
1168/1168 [==============================] - 0s 72us/step - loss: 12332.5593 - mean_absolute_error: 12332.5576 - val_loss: 18008.6102 - val_mean_absolute_error: 18008.6113
Epoch 311/500
1168/1168 [==============================] - 0s 69us/step - loss: 12343.6882 - mean_absolute_error: 12343.6885 - val_loss: 17998.7063 - val_mean_absolute_error: 17998.7070
Epoch 312/500
1168/1168 [==============================] - 0s 71us/step - loss: 12323.6890 - mean_absolute_error: 12323.6904 - val_loss: 17991.1552 - val_mean_absolute_error: 17991.1543
Epoch 313/500
1168/1168 [==============================] - 0s 69us/step - loss: 1232

1168/1168 [==============================] - 0s 71us/step - loss: 12174.6088 - mean_absolute_error: 12174.6084 - val_loss: 17850.0784 - val_mean_absolute_error: 17850.0801
Epoch 353/500
1168/1168 [==============================] - 0s 73us/step - loss: 12304.2209 - mean_absolute_error: 12304.2197 - val_loss: 17847.8940 - val_mean_absolute_error: 17847.8945
Epoch 354/500
1168/1168 [==============================] - 0s 82us/step - loss: 12214.9984 - mean_absolute_error: 12214.9961 - val_loss: 17898.3439 - val_mean_absolute_error: 17898.3418
Epoch 355/500
1168/1168 [==============================] - 0s 80us/step - loss: 12184.3467 - mean_absolute_error: 12184.3486 - val_loss: 17842.9601 - val_mean_absolute_error: 17842.9609
Epoch 356/500
1168/1168 [==============================] - 0s 76us/step - loss: 12195.5473 - mean_absolute_error: 12195.5479 - val_loss: 17853.8488 - val_mean_absolute_error: 17853.8477
Epoch 357/500
1168/1168 [==============================] - 0s 72us/step - loss: 1219

1168/1168 [==============================] - 0s 79us/step - loss: 11995.9460 - mean_absolute_error: 11995.9463 - val_loss: 17818.1041 - val_mean_absolute_error: 17818.1035
Epoch 397/500
1168/1168 [==============================] - 0s 77us/step - loss: 12012.6442 - mean_absolute_error: 12012.6445 - val_loss: 17861.1290 - val_mean_absolute_error: 17861.1270
Epoch 398/500
1168/1168 [==============================] - 0s 71us/step - loss: 11986.8273 - mean_absolute_error: 11986.8271 - val_loss: 17754.7408 - val_mean_absolute_error: 17754.7422
Epoch 399/500
1168/1168 [==============================] - 0s 71us/step - loss: 11978.6934 - mean_absolute_error: 11978.6943 - val_loss: 17759.6787 - val_mean_absolute_error: 17759.6777
Epoch 400/500
1168/1168 [==============================] - 0s 73us/step - loss: 11946.5019 - mean_absolute_error: 11946.5039 - val_loss: 17712.8052 - val_mean_absolute_error: 17712.8047
Epoch 401/500
1168/1168 [==============================] - 0s 73us/step - loss: 1192

1168/1168 [==============================] - 0s 75us/step - loss: 11753.4894 - mean_absolute_error: 11753.4873 - val_loss: 17646.1194 - val_mean_absolute_error: 17646.1172
Epoch 441/500
1168/1168 [==============================] - 0s 72us/step - loss: 11775.3260 - mean_absolute_error: 11775.3281 - val_loss: 17676.6697 - val_mean_absolute_error: 17676.6680
Epoch 442/500
1168/1168 [==============================] - 0s 72us/step - loss: 11720.5225 - mean_absolute_error: 11720.5244 - val_loss: 17743.5897 - val_mean_absolute_error: 17743.5898
Epoch 443/500
1168/1168 [==============================] - 0s 73us/step - loss: 11752.1235 - mean_absolute_error: 11752.1240 - val_loss: 17621.7291 - val_mean_absolute_error: 17621.7285
Epoch 444/500
1168/1168 [==============================] - 0s 71us/step - loss: 11731.6038 - mean_absolute_error: 11731.6025 - val_loss: 17626.6334 - val_mean_absolute_error: 17626.6348
Epoch 445/500
1168/1168 [==============================] - 0s 73us/step - loss: 1172

1168/1168 [==============================] - 0s 73us/step - loss: 11519.8313 - mean_absolute_error: 11519.8320 - val_loss: 17579.6064 - val_mean_absolute_error: 17579.6035
Epoch 485/500
1168/1168 [==============================] - 0s 71us/step - loss: 11567.9464 - mean_absolute_error: 11567.9482 - val_loss: 17572.6263 - val_mean_absolute_error: 17572.6270
Epoch 486/500
1168/1168 [==============================] - 0s 73us/step - loss: 11586.3086 - mean_absolute_error: 11586.3096 - val_loss: 17596.4968 - val_mean_absolute_error: 17596.4961
Epoch 487/500
1168/1168 [==============================] - 0s 70us/step - loss: 11548.2333 - mean_absolute_error: 11548.2334 - val_loss: 17583.0552 - val_mean_absolute_error: 17583.0566
Epoch 488/500
1168/1168 [==============================] - 0s 73us/step - loss: 11503.9598 - mean_absolute_error: 11503.9619 - val_loss: 17536.7306 - val_mean_absolute_error: 17536.7305
Epoch 489/500
1168/1168 [==============================] - 0s 73us/step - loss: 1149

In [28]:
y_hat = model.predict(X)

In [29]:
y_hat[:5]

array([[207971.92],
       [196107.61],
       [210215.77],
       [165713.92],
       [287496.62]], dtype=float32)

In [30]:
train.iloc[:5]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [31]:
lr.fit(X,y)
y_hat_lr = lr.predict(X)

In [32]:
y_hat_lr[:5]

array([207659.14761438, 205992.9069924 , 205791.63040861, 165907.52281102,
       290786.73189639])

In [33]:
test = pd.read_csv(r'.\kaggle\housing\house-prices-advanced-regression-techniques\test.csv')

In [34]:
test.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [35]:
test_preproc = full_Pipeline.fit_transform(test)

In [ ]:
test_X.shape